## 57. Insert Interval
- Description:
  <blockquote>
    You are given an array of non-overlapping intervals `intervals` where `intervals[i] = [starti, endi]` represent the start and the end of the `ith` interval and `intervals` is sorted in ascending order by `starti`. You are also given an interval `newInterval = [start, end]` that represents the start and end of another interval.
     
    Insert `newInterval` into `intervals` such that `intervals` is still sorted in ascending order by `starti` and `intervals` still does not have any overlapping intervals (merge overlapping intervals if necessary).
     
    Return `intervals` *after the insertion* .
     
    **Note** that you don't need to modify `intervals` in-place. You can make a new array and return it.
     
    **Example 1:**
    **Input:** intervals = `[1, 3],[6,9]`, newInterval = [2,5]
    **Output:** `[1, 5],[6,9]`
     
    **Example 2:**
    **Input:** intervals = `[1, 2],[3,5],[6,7],[8,10],[12,16]`, newInterval = [4,8]
    **Output:** `[1, 2],[3,10],[12,16]`
    **Explanation:** Because the new interval [4,8] overlaps with [3,5],[6,7],[8,10].
     
    **Constraints:**
     
    - `0 <= intervals.length <= 104`
    - `intervals[i].length == 2`
    - `0 <= starti<= endi<= 105`
    - `intervals` is sorted by `starti` in **ascending** order.
    - `newInterval.length == 2`
    - `0 <= start <= end <= 105`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/insert-interval/description/)

- Topics: Array, Intervals

- Difficulty: Easy / Medium

- Resources: example_resource_URL

### Solution 1, Linear Search with single for loop
Solution description
- Time Complexity: O(N)
- Space Complexity: O(1)

In [ ]:
class Solution:
    def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
        result = []

        for idx, curr_interval in enumerate(intervals):
            # the new interval is after the of curr_interval,
            # New interval start is after current interval end
            # so we can leave the current interval as is because the new one does not overlap with it
            if newInterval[0] > curr_interval[1]:
                result.append(curr_interval)
            # the new interval is before the curr_interval,
            # New interval end is before current interval start
            # so we can add the new interval and append the remaining intervals directly as they are already sorted
            elif newInterval[1] < curr_interval[0]:
                result.append(newInterval)
                # newInterval = curr_interval
                return result + intervals[idx:]
            # the new interval overlaps the curr_interval, so we must choose the min for start and max for end of interval
            else:
                newInterval[0] = min(newInterval[0], curr_interval[0])
                newInterval[1] = max(newInterval[1], curr_interval[1])

        result.append(newInterval)

        return result

### Solution 2, Linear Search with three while loops
Let N be the number of intervals.
- Time Complexity: O(N)
  - We iterate through the intervals once, and each interval is considered and processed only once.
- Space Complexity: O(1)
  - We only use the result (res) array to store output, so this could be considered O(1).

In [ ]:
class Solution:
    def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
        n = len(intervals)
        i = 0
        res = []

        # Case 1: No overlapping before merging intervals
        # When the endpoint of the current interval (intervals[i][1]) is less than the starting point of the new interval
        while i < n and intervals[i][1] < newInterval[0]:
            res.append(intervals[i])
            i += 1

        # Case 2: Overlapping and merging intervals
        # When the starting point of the current interval (intervals[i][0]) is less than or equal to the ending point of the new interval (newInterval[1]), 
        # the above while loop already guarantees that the ending point of the current interval is also greater than or equal to the starting point of the new interval
        # indicating an overlap
        while i < n and intervals[i][0] <= newInterval[1]:
            newInterval[0] = min(newInterval[0], intervals[i][0])
            newInterval[1] = max(newInterval[1], intervals[i][1])
            i += 1
        res.append(newInterval)

        # Case 3: No overlapping after merging newInterval
        while i < n:
            res.append(intervals[i])
            i += 1

        return res

### Solution 3, Binary Search to find insertion point
Let N be the number of intervals.

    Time complexity: O(N)

    The binary search for finding the position to insert the newInterval has a time complexity of O(logN). However, the insertion of the newInterval into the list may take O(N) time in the worst case, as it could involve shifting elements within the list. Consequently, the overall time complexity is O(N+logN), which simplifies to O(N).

    Space complexity: O(N)

    We use the additional space to store the result (res) and perform calculations using res, so it does count towards the space complexity. In the worst case, the size of res will be proportional to the number of intervals in the input list.


In [ ]:
class Solution:
    def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
        # If the intervals vector is empty, return a vector containing the newInterval
        if not intervals:
            return [newInterval]

        n = len(intervals)
        target = newInterval[0]
        left, right = 0, n - 1

        # Binary search to find the position to insert newInterval
        while left <= right:
            mid = (left + right) // 2
            if intervals[mid][0] < target:
                left = mid + 1
            else:
                right = mid - 1

        # Insert newInterval at the found position
        intervals.insert(left, newInterval)

        # Merge overlapping intervals
        res = []
        for interval in intervals:
            # If res is empty or there is no overlap, add the interval to the result
            if not res or res[-1][1] < interval[0]:
                res.append(interval)
            # If there is an overlap, merge the intervals by updating the end of the last interval in res
            else:
                res[-1][1] = max(res[-1][1], interval[1])
        return res

### Solution 4, Heap
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
class Solution:
    def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
        heap = []
        result = []

        # add start & end to heap (-1 is start, 1 is end)
        for interval_start, interval_end in intervals + [newInterval]:
            heapq.heappush(heap, (interval_start, -1))
            heapq.heappush(heap, (interval_end, 1))

        cur = 0
        start = None

        print(heap)

        while heap:
            interval_no, val = heapq.heappop(heap)       # pop heap
            print(f"interval_no = {interval_no}, val = {val}")

            if start is None:
                # is start is None, assign interval_no to start (interval start)
                start = interval_no

            cur += val                         # keep counting until close interval

            if cur==0:                        # when cur == 0, meaning we can close the interval
                result.append([start, interval_no])             # append interval to result
                start = None                       # reset s to None

        return result